###### Imports and Settings

In [2]:
import pandas as pd
import numpy as np
import requests
#import io
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None  # default='warn'
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces

# Decennial SF1 2010

In [3]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [4]:
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

### SF1

In [5]:
dataguide = pd.read_csv('../../Data Guides/DATA GUIDE SF12010.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [6]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
dg2 = dataguide[dataguide['ID'].between(47, 92)]

In [7]:
dfs = [dg1, dg2]
droppers = ['ID', 'Category', 'Sub Category']
for df in dfs:
    df.drop(droppers, axis = 1, inplace = True)

In [10]:
url_str= 'https://api.census.gov/data/2010/dec/sf1?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
tail_cols3 = 'GeoFIPS2'

In [11]:
splits = ['60280', '63140', '48980', '80200', '73460', '29920', '70580']

In [9]:
results = []
for df in dfs:
    dataguide = df
    for col_name, col_data in df.items():
        var_list = list(dataguide['Variable']) #make variables list
        var_list = deque(var_list)
        var_list.appendleft(head2)
        var_list.appendleft(head1)
        col_list = list(dataguide['Column Name']) #make columns list
        col_list.append(tail_cols1)
        col_list.append(tail_cols2)
        col_list = deque(col_list)
        col_list.appendleft(head2)
        col_list.appendleft(head1)
        predicates= {} #call for all counties in state of TN
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "county:*"
        predicates["in"]= "state:47"                                                             
        data = requests.get(url_str, params= predicates)                                                                
        col_names = col_list
        df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        #df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
        predicates= {} #call for all counties in state of KY
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "county:*"
        predicates["in"]= "state:21"                                                             
        data = requests.get(url_str, params= predicates)                                                                
        col_names = col_list
        kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
        df = pd.concat([df, kycos], axis = 0)
        predicates= {} #call for all places in TN
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "place:*"
        predicates["in"]= "state:47"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        places=places.loc[places['GEO_ID'].isin(censusplaces)] #filter for places in the region
        df = pd.concat([df, places], axis = 0)
        predicates= {} #call for all places in KY
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "place:*"
        predicates["in"]= "state:21"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        places=places.loc[places['GEO_ID'].isin(censusplaces)] #filter for places of concern in KY
        df = pd.concat([df, places], axis = 0)
        col_list.remove(tail_cols2) #adjust the column list for different formats of geos
        predicates= {} #call for state of TN
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "state:47"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
        df = pd.concat([df, state], axis = 0)
        predicates= {} #call for US
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "us:*"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
        df = pd.concat([df, national], axis = 0)
        results.append(df)
new_df = pd.concat(results, axis = 1)
new_df = new_df.transpose().reset_index(drop = False).drop_duplicates()
new_df.columns = new_df.iloc[0]
new_df = new_df.set_index('NAME').transpose().drop(columns = ['StateFIPS', 'GeoFIPS']).reset_index(drop = True)
new_df = new_df.rename_axis(None, axis = 1)
print('Okay Finished')

Okay Finished


In [12]:
part_one = new_df

In [13]:
#this works
data_appended = []
for df in dfs:
    dataguide = df
    for col_name, col_data in df.items():
        var_list = list(dataguide['Variable']) #make variables list
        var_list = deque(var_list)
        var_list.appendleft(head2)
        var_list.appendleft(head1)
        col_list = list(dataguide['Column Name']) #make columns list
        col_list.append(tail_cols1)
        col_list.append(tail_cols2)
        col_list.append(tail_cols3)
        col_list = deque(col_list)
        col_list.appendleft(head2)  
        col_list.appendleft(head1)
        temp = []
        for thingy in splits:
            predicates= {}
            get_vars= var_list
            predicates["get"]= ",". join(get_vars)
            predicates["for"]= 'county (or part):*'
            predicates["in"] = "state:47, place:{}".format(thingy)
            data = requests.get(url_str, params= predicates)
            col_names = col_list
            data = pd.DataFrame(columns=col_names, data=data.json()[1:])
            temp.append(data) #append the current instance to the last
        temp = pd.concat(temp)
        thing = temp
    data_appended.append(temp)
new_df = pd.concat(data_appended, axis = 1) 
new_df = new_df.transpose().reset_index(drop = False).drop_duplicates()
new_df.columns = new_df.iloc[0]
new_df = new_df.set_index('NAME').transpose().drop(columns = ['StateFIPS', 'GeoFIPS', 'GeoFIPS2']).reset_index(drop = True)
new_df = new_df.rename_axis(None, axis = 1)


    
print('Your API call is complete')

Your API call is complete


In [15]:
new_df.head(2)

,NAME,GEO_ID,pop,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces_alone,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,hhsize_avg,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,tenure_total_series,tenure_owneroccunits_mortgage,tenure_owneroccunits_nomortgage,tenure_renteroccunits,hhtype_total_series,hhtype_familyhh,hhtype_familyhh_marriedcouplefam,hhtype_familyhh_otherfam,hhtype_familyhh_malenospouse,hhtype_familyhh_femalenospouse,hhtype_nonfamhh,hhtype_nonfamhh_householderalone,hhtype_nonfamhh_householdernotalone
0,"Robertson County (part), Portland city, Tennessee",1550000US4760280147,57,57,31,0,3,3,1,1,0,1,0,1,1,2,1,4,4,2,2,1,1,1,1,0,1,0,26,0,0,2,2,0,0,0,0,5,0,1,1,3,2,2,2,2,1,1,0,2,0,0,57,52,2,2,0,0,1,0,51,2,2.59,27,27,22,5,22,9,6,7,22,19,16,3,2,1,3,1,2
1,"Sumner County (part), Portland city, Tennessee",1550000US4760280165,11423,11423,5546,522,515,487,220,161,74,57,221,426,433,431,382,371,304,254,94,121,77,97,119,85,68,27,5877,466,455,432,246,141,67,78,239,496,428,456,399,393,351,280,106,151,105,129,160,128,92,79,11423,10553,399,33,62,6,175,195,10336,450,2.74,4597,4597,4161,436,4161,2150,627,1384,4161,3040,2159,881,239,642,1121,882,239


In [16]:
balances = new_df

In [17]:
balances['GEO_ID'] = '1'

In [25]:
msas = ['34980', '17300']

In [28]:
#this works msa
data_appended = []
for df in dfs:
    dataguide = df
    for col_name, col_data in df.items():
        var_list = list(dataguide['Variable']) #make variables list
        var_list = deque(var_list)
        var_list.appendleft(head2)
        var_list.appendleft(head1)
        col_list = list(dataguide['Column Name']) #make columns list
        col_list.append(tail_cols1)
        col_list = deque(col_list)
        col_list.appendleft(head2)  
        col_list.appendleft(head1)
        temp = []
        for msa in msas:
            predicates= {}
            get_vars= var_list
            predicates["get"]= ",". join(get_vars)
            predicates["for"]= "metropolitan statistical area/micropolitan statistical area:{}".format(msa)
            data = requests.get(url_str, params= predicates)
            col_names = col_list
            data = pd.DataFrame(columns=col_names, data=data.json()[1:])
            temp.append(data) #append the current instance to the last
        temp = pd.concat(temp)
        thing = temp
    data_appended.append(temp)
new_df = pd.concat(data_appended, axis = 1) 
new_df = new_df.transpose().reset_index(drop = False).drop_duplicates()
new_df.columns = new_df.iloc[0]
new_df = new_df.set_index('NAME').transpose().drop(columns = ['StateFIPS']).reset_index(drop = True)
new_df = new_df.rename_axis(None, axis = 1)


    
print('Your API call is complete')

Your API call is complete


In [30]:
msas = new_df

In [31]:
dfs = [part_one, balances, new_df]

In [32]:
data = pd.concat(dfs)

In [33]:
data = data.set_index(['NAME', 'GEO_ID']).transpose()
data.head()

NAME,"Washington County, Tennessee","Williamson County, Tennessee","Wayne County, Tennessee","Weakley County, Tennessee","Wilson County, Tennessee","Grainger County, Tennessee","Hamilton County, Tennessee","Hawkins County, Tennessee","Haywood County, Tennessee","Lake County, Tennessee","Hickman County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Jackson County, Tennessee","Jefferson County, Tennessee","Johnson County, Tennessee","Knox County, Tennessee","Hardin County, Tennessee","Henry County, Tennessee","Marshall County, Tennessee","Lauderdale County, Tennessee","Lawrence County, Tennessee","Lewis County, Tennessee","Lincoln County, Tennessee","Loudon County, Tennessee","McMinn County, Tennessee","Macon County, Tennessee","McNairy County, Tennessee","Madison County, Tennessee","Marion County, Tennessee","Morgan County, Tennessee","Maury County, Tennessee","Meigs County, Tennessee","Monroe County, Tennessee","Montgomery County, Tennessee","Moore County, Tennessee","Scott County, Tennessee","Obion County, Tennessee","Overton County, Tennessee","Perry County, Tennessee","Pickett County, Tennessee","Polk County, Tennessee","Putnam County, Tennessee","Rhea County, Tennessee","Roane County, Tennessee","Rutherford County, Tennessee","Robertson County, Tennessee","Sullivan County, Tennessee","Sequatchie County, Tennessee","Sevier County, Tennessee","Shelby County, Tennessee","Smith County, Tennessee","Stewart County, Tennessee","Van Buren County, Tennessee","Warren County, Tennessee","White County, Tennessee","Sumner County, Tennessee","Tipton County, Tennessee","Trousdale County, Tennessee","Unicoi County, Tennessee","Union County, Tennessee","Cheatham County, Tennessee","Bledsoe County, Tennessee","Blount County, Tennessee","Bradley County, Tennessee","Carter County, Tennessee","Carroll County, Tennessee","Anderson County, Tennessee","Bedford County, Tennessee","Benton County, Tennessee","Campbell County, Tennessee","Cannon County, Tennessee","Crockett County, Tennessee","Chester County, Tennessee","Claiborne County, Tennessee","Clay County, Tennessee","Cocke County, Tennessee","Coffee County, Tennessee","Dyer County, Tennessee","Cumberland County, Tennessee","Davidson County, Tennessee","Decatur County, Tennessee","DeKalb County, Tennessee","Dickson County, Tennessee","Fayette County, Tennessee","Fentress County, Tennessee","Franklin County, Tennessee","Gibson County, Tennessee","Giles County, Tennessee","Greene County, Tennessee","Grundy County, Tennessee","Hamblen County, Tennessee","Hancock County, Tennessee","Hardeman County, Tennessee","Henderson County, Tennessee","Simpson County, Kentucky","Allen County, Kentucky","Nolensville town, Tennessee","Adams city, Tennessee","Ashland City town, Tennessee","Belle Meade city, Tennessee","Oak Hill city, Tennessee","Orlinda city, Tennessee","Pegram town, Tennessee","Portland city, Tennessee","Pleasant View city, Tennessee","Ridgetop city, Tennessee","Berry Hill city, Tennessee","Brentwood city, Tennessee","Burns town, Tennessee","Hendersonville city, Tennessee","Kingston Springs town, Tennessee","Lafayette city, Tennessee","La Vergne city, Tennessee","Lebanon city, Tennessee","McEwen city, Tennessee","Millersville city, Tennessee","Mitchellville city, Tennessee","Mount Juliet city, Tennessee","Mount Pleasant city, Tennessee","Murfreesboro city, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee","New Johnsonville city, Tennessee","Vanleer town, Tennessee","Watertown city, Tennessee","Waverly city, Tennessee","Westmoreland town, Tennessee","White Bluff town, Tennessee","White House city, Tennessee","Eagleville city, Tennessee","Erin city, Tennessee","Fairview city, Tennessee","Forest Hills city, Tennessee","Franklin city, Tennessee","Gallatin city, Tennessee","Goodlettsville city, Tennessee","Greenbrier town, Tennessee","Cedar Hill city, Tennessee","Charlotte town, Tennessee","Clarksville city, Tennessee","Coopertown town, Tennessee","

In [20]:
numcols = list(data.columns)
numcols
data[numcols] = data[numcols].astype(float)

In [21]:
GNRCCounties = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                data[('Rutherford County, Tennessee', '0500000US47149')]]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                   data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                   data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                   data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                   data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                   data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                   data[('Rutherford County, Tennessee', '0500000US47149')],data[('Maury County, Tennessee', '0500000US47119')]]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
               data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
               data[('Williamson County, Tennessee', '0500000US47187')],data[('Rutherford County, Tennessee', '0500000US47149')],
               data[('Maury County, Tennessee', '0500000US47119')]]
data['MPO'] = sum(MPOCounties)
#Cheatham
CheathInc = [data[('Ashland City town, Tennessee', '1600000US4702180')],data[('Kingston Springs town, Tennessee', '1600000US4739660')],
             data[('Pegram town, Tennessee', '1600000US4757480')],data[('Pleasant View city, Tennessee', '1600000US4759560')]]
data[('Cheatham Incorporated', 'None')] = sum(CheathInc)
data[('Cheatham Unincorporated', 'None')] = data[('Cheatham County, Tennessee', '0500000US47021')] - data[('Cheatham Incorporated', 'None')]
#Dickson
DicksInc = [data[('Burns town, Tennessee', '1600000US4709880')],data[('Charlotte town, Tennessee', '1600000US4713080')],
            data[('Dickson city, Tennessee', '1600000US4720620')],data[('Slayden town, Tennessee', '1600000US4769080')],
            data[('Vanleer town, Tennessee', '1600000US4776860')],data[('White Bluff town, Tennessee', '1600000US4779980')]]
data[('Dickson Incorporated', 'None')] = sum(DicksInc)
data[('Dickson Unincorporated', 'None')] = data[('Dickson County, Tennessee', '0500000US47043')] - data[('Dickson Incorporated', 'None')]
#Houston
HoustInc = [data[('Erin city, Tennessee', '1600000US4724320')],data[('Houston County (part), Tennessee Ridge town, Tennessee', '1')]]
data[('Houston Incorporated', 'None')] = sum(HoustInc)
data[('Houston Unincorporated', 'None')] = data[('Houston County, Tennessee', '0500000US47083')] - data[('Houston Incorporated', 'None')]
#Humphreys
HumphInc = [data[('McEwen city, Tennessee', '1600000US4744840')],data[('New Johnsonville city, Tennessee', '1600000US4752820')],
            data[('Waverly city, Tennessee', '1600000US4778560')]]
data[('Humphreys Incorporated', 'None')] = sum(HumphInc)
data[('Humphreys Unincorporated', 'None')] = data[('Humphreys County, Tennessee', '0500000US47085')] - data[('Humphreys Incorporated', 'None')]
#Maury
MauryInc = [data[('Mount Pleasant city, Tennessee', '1600000US4751080')],data[('Columbia city, Tennessee', '1600000US4716540')],
            data[('Maury County (part), Spring Hill city, Tennessee', '1')]]
data[('Maury Incorporated', 'None')] = sum(MauryInc)
data[('Maury Unincorporated', 'None')] = data[('Maury County, Tennessee', '0500000US47119')] - data[('Maury Incorporated', 'None')]
#Montgomery
data[('Montgomery Incorporated', 'None')] = data[('Clarksville city, Tennessee', '1600000US4715160')]
data[('Montgomery Unincorporated', 'None')] = data[('Montgomery County, Tennessee', '0500000US47125')] - data[('Montgomery Incorporated', 'None')]
#Robertson
RobInc = [data[('Adams city, Tennessee', '1600000US4700200')],data[('Greenbrier town, Tennessee', '1600000US4730960')],
          data[('Cedar Hill city, Tennessee', '1600000US4711980')],data[('Coopertown town, Tennessee', '1600000US4716980')], 
         data[('Cross Plains city, Tennessee', '1600000US4718420')],data[('Springfield city, Tennessee', '1600000US4770500')], 
          data[('Orlinda city, Tennessee', '1600000US4756020')],
         data[('Robertson County (part), Portland city, Tennessee', '1')],data[('Robertson County (part), Millersville city, Tennessee', '1')], 
         data[('Robertson County (part), White House city, Tennessee', '1')], data[('Robertson County (part), Ridgetop city, Tennessee', '1')]]
data[('Robertson Incorporated', 'None')] = sum(RobInc)
data[('Robertson Unincorporated', 'None')] = data[('Robertson County, Tennessee', '0500000US47147')] - data[('Robertson Incorporated', 'None')]
#Rutherford
RuthInc = [data[('Eagleville city, Tennessee', '1600000US4722360')],data[('La Vergne city, Tennessee', '1600000US4741200')],
           data[('Murfreesboro city, Tennessee', '1600000US4751560')],data[('Smyrna town, Tennessee', '1600000US4769420')]]
data[('Rutherford Incorporated', 'None')] = sum(RuthInc)
data[('Rutherford Unincorporated', 'None')] = data[('Rutherford County, Tennessee', '0500000US47149')] - data[('Rutherford Incorporated', 'None')]
#Stewart
data[('Stewart County (part), Tennessee Ridge town, Tennessee', '1')] = data[('Tennessee Ridge town, Tennessee', '1600000US4773460')] - data[('Houston County (part), Tennessee Ridge town, Tennessee', '1')]
StewInc = [data[('Cumberland City town, Tennessee', '1600000US4718820')],data[('Dover city, Tennessee', '1600000US4721400')],
            data[('Stewart County (part), Tennessee Ridge town, Tennessee', '1')]]
data[('Stewart Incorporated', 'None')] = sum(StewInc)
data[('Stewart Unincorporated', 'None')] = data[('Stewart County, Tennessee', '0500000US47161')] - data[('Stewart Incorporated', 'None')]
#Sumner
SumInc = [data[('Hendersonville city, Tennessee', '1600000US4733280')],data[('Mitchellville city, Tennessee', '1600000US4749460')],
          data[('Westmoreland town, Tennessee', '1600000US4779420')],data[('Gallatin city, Tennessee', '1600000US4728540')], 
         data[('Sumner County (part), Portland city, Tennessee', '1')],data[('Sumner County (part), Millersville city, Tennessee', '1')], 
         data[('Sumner County (part), White House city, Tennessee', '1')],data[('Sumner County (part), Goodlettsville city, Tennessee', '1')]]
data[('Sumner Incorporated', 'None')] = sum(SumInc)
data[('Sumner Unincorporated', 'None')] = data[('Sumner County, Tennessee', '0500000US47165')] - data[('Sumner Incorporated', 'None')]
#Williamson
WilliInc = [data[('Nolensville town, Tennessee', '1600000US4753460')],data[('Brentwood city, Tennessee', '1600000US4708280')],
          data[('Fairview city, Tennessee', '1600000US4725440')],data[('Franklin city, Tennessee', '1600000US4727740')], 
         data[("Thompson's Station town, Tennessee", '1600000US4773900')],data[('Williamson County (part), Spring Hill city, Tennessee', '1')]]
data[('Williamson Incorporated', 'None')] = sum(WilliInc)
data[('Williamson Unincorporated', 'None')] = data[('Williamson County, Tennessee', '0500000US47187')] - data[('Williamson Incorporated', 'None')]
#Wilson
WilsonInc = [data[('Lebanon city, Tennessee', '1600000US4741520')],data[('Mount Juliet city, Tennessee', '1600000US4750780')],
             data[('Watertown city, Tennessee', '1600000US4778320')]]
data[('Wilson Incorporated', 'None')] = sum(WilsonInc)
data[('Wilson Unincorporated', 'None')] = data[('Wilson County, Tennessee', '0500000US47189')] - data[('Wilson Incorporated', 'None')]

In [22]:
data = data.transpose().reset_index()
data.head()

,NAME,GEO_ID,pop,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,raceeth_total_series,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces_alone,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,hhsize_avg,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,tenure_total_series,tenure_owneroccunits_mortgage,tenure_owneroccunits_nomortgage,tenure_renteroccunits,hhtype_total_series,hhtype_familyhh,hhtype_familyhh_marriedcouplefam,hhtype_familyhh_otherfam,hhtype_familyhh_malenospouse,hhtype_familyhh_femalenospouse,hhtype_nonfamhh,hhtype_nonfamhh_householderalone,hhtype_nonfamhh_householdernotalone
0,"Washington County, Tennessee",0500000US47179,122979.0,122979.0,60089.0,3396.0,3455.0,3650.0,2195.0,1964.0,998.0,1003.0,2840.0,3946.0,3758.0,4112.0,4099.0,4393.0,4308.0,3995.0,1582.0,2167.0,1255.0,1687.0,2076.0,1497.0,959.0,754.0,62890.0,3270.0,3438.0,3382.0,2041.0,2153.0,1134.0,1165.0,2958.0,4055.0,3699.0,4061.0,4027.0,4434.0,4488.0,4130.0,1588.0,2334.0,1301.0,1797.0,2402.0,1901.0,1546.0,1586.0,122979.0,112619.0,4814.0,377.0,1466.0,38.0,1534.0,2131.0,110902.0,3633.0,2.31,57254.0,57254.0,51322.0,5932.0,51322.0,21483.0,12487.0,17352.0,51322.0,32319.0,24570.0,7749.0,2244.0,5505.0,19003.0,15326.0,3677.0
1,"Williamson County, Tennessee",0500000US47187,183182.0,183182.0,89336.0,6324.0,7997.0,8372.0,4712.0,2098.0,684.0,621.0,2155.0,3682.0,4755.0,6623.0,7440.0,7677.0,7402.0,6010.0,2131.0,2793.0,1405.0,1693.0,1930.0,1297.0,893.0,642.0,93846.0,6039.0,7718.0,7870.0,4597.0,1705.0,528.0,607.0,2188.0,4177.0,5558.0,7285.0,8076.0,8621.0,7767.0,6229.0,2141.0,2793.0,1485.0,1770.0,2202.0,1715.0,1330.0,1445.0,183182.0,163728.0,7941.0,396.0,5517.0,82.0,2784.0,2734.0,158769.0,8166.0,2.81,68498.0,68498.0,64886.0,3612.0,64886.0,40747.0,11970.0,12169.0,64886.0,51242.0,44172.0,7070.0,1734.0,5336.0,13644.0,11505.0,2139.0
2,"Wayne County, Tennessee",0500000US47181,17021.0,17021.0,9402.0,456.0,458.0,502.0,377.0,225.0,95.0,116.0,407.0,739.0,793.0,719.0,767.0,734.0,697.0,564.0,220.0,333.0,188.0,241.0,325.0,205.0,142.0,99.0,7619.0,389.0,403.0,454.0,295.0,179.0,98.0,89.0,218.0,384.0,415.0,438.0,524.0,596.0,559.0,533.0,230.0,317.0,170.0,241.0,376.0,288.0,212.0,211.0,17021.0,15712.0,978.0,54.0,30.0,5.0,80.0,162.0,15535.0,277.0,2.43,7287.0,7287.0,6136.0,1151.0,6136.0,2496.0,2401.0,1239.0,6136.0,4303.0,3337.0,966.0,325.0,641.0,1833.0,1629.0,204.0
3,"Weakley County, Tennessee",0500000US47183,35021.0,35021.0,17159.0,955.0,961.0,1041.0,653.0,893.0,529.0,539.0,1125.0,1025.0,872.0,942.0,1046.0,1109.0,1123.0,1085.0,400.0,573.0,352.0,426.0,573.0,442.0,278.0,217.0,17862.0,929.0,901.0,986.0,604.0,1041.0,552.0,499.0,870.0,971.0,879.0,955.0,1051.0,1213.0,1159.0,1112.0,471.0,611.0,348.0,463.0,703.0,564.0,458.0,522.0,35021.0,31124.0,2736.0,101.0,368.0,8.0,200.0,484.0,30696.0,700.0,2.36,15495.0,15495.0,13898.0,1597.0,13898.0,5204.0,3904.0,4790.0,13898.0,8836.0,6634.0,2202.0,621.0,1581.0,5062.0,3922.0,1140.0
4,"Wilson County, Tennessee",0500000US47189,113993.0,113993.0,55834.0,3746.0,4118.0,4271.0,2473.0,1480.0,684.0,534.0,1590.0,2872.0,3382.0,4097.0,4273.0,4663.0,4373.0,3739.0,1374.0,1924.0,1062.0,1406.0,1731.0,1061.0,566.0,415.0,58159.0,3572.0,3933.0,4086.0,2362.0,1273.0,554.0,543.0,1639.0,3158.0,3695.0,4283.0,4501.0,

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 80 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   NAME                                              197 non-null    object 
 1   GEO_ID                                            197 non-null    object 
 2   pop                                               197 non-null    float64
 3   agebysex_total_series                             197 non-null    float64
 4   age_total_male                                    197 non-null    float64
 5   age_m_u5                                          197 non-null    float64
 6   age_m_5to9                                        197 non-null    float64
 7   age_m_10to14                                      197 non-null    float64
 8   age_m_15to17                                      197 non-null    float64
 9   age_m_18to19         

In [24]:
data.to_feather('../../Raw Data/SF12010')